[GoogleTest]: https://github.com/google/googletest
# スパイ(Spying)とモック(Mocking)

[GoogleTest][]を使って、
[テスト駆動開発による組み込みプログラミング
――C言語とオブジェクト指向で学ぶアジャイルな設計](https://www.oreilly.co.jp/books/9784873116143/)
を実践に応用しました。

テストにおけるスパイとモックとは、以下のような役割のものと解釈しました。  
※個人的な解釈ですので、厳密には間違っている可能性があります。

### スパイ(Spying)
* テスト対象のオブジェクト(Object Under Test:OUT)に渡されたパラメタを記録して、意図した値・意図した頻度で
パラメタが渡されているかを記録します。
* テスト対象のオブジェクト(OUT)内部で保持するデータ構造が意図した通りに操作されているか、
意図したデータが記録されているかを確認します。

![Spying](images/Spying.png)

### モック(Mocking)
* テスト対象のオブジェクト(OUT)が、他のオブジェクトに依存している場合、
そのオブジェクトの代用品として動作します。
* テスト対象のオブジェクト(OUT)が、他のオブジェクトを意図した通りに呼び出しているか、
他のオブジェクトから出力された結果を意図通りに操作しているかを確認します。

![Mocking](images/Mocking.png)


# Spying

Spy の利用においては、Spy によるテストを踏まえて、テスト対象コードの実装方法についても考慮が必要です。
以下にテスト対象コードの実装方法について注意点を記します。

#### OUT内部データ構造へのアクセス
* `*.c` に構造体定義を置かないこと。 
* private な構造体は `*_private.h` などに定義し、本実装では `#include` の呼び出し元を制限しておくこと。

こうすることで、Spyコードが内部データに対してアクセスできます。

#### OUT内部関数の Wrapping
* 多段 spy が出来るように、OUT内のプライベート関数は、関数テーブルからの呼び出しにすること

こうすることで、OUT内のプライベート関数も Spy 関数による置き換えができるようになり、
OUTの挙動を確認できます。


# テストを考慮した実装例

以下のように、メソッドと内部データを保持するための構造体を定義していたとします。

```C:hoge_private.h
typedef struct hoge_struct {
    void (*inc_x)(struct hoge_struct *self, int x);
    void (*inc_10)(struct hoge_struct *self);
    bool (*ge_x)(struct hoge_struct *self, int x);
    
    int total;
} *hoge;
```


# テストを考慮した実装例(cont'd)

この hoge モジュールを以下のように実装していたとします。

```C:hoge.c
// private definitions
#include "hoge_private.h"

// prototypes of private functions
static void inc_x(struct hoge_struct *self, int x);
static void inc_10(struct hoge_struct *self);
static bool ge_x(struct hoge_struct *self, int x);

// public functions
void *
hoge_new(void)
{
    void *hoge;
    hoge = calloc(1, sizeof(struct hoge_struct));
    if (!hoge) {
        // error message
        return NULL;
    }
    hoge->inc_x = inc_x;
    hoge->inc_10 = inc_10;
    hoge->ge_x = ge_x;
    hoge->total = 0;
    return hoge;
}

// private functions
static void
inc_x(struct hoge_struct *self, int x)
{
    self->total += x;
}

static void
inc_10(struct hoge_struct *self)
{
    self->total += 10;
}

static bool
ge_x(struct hoge_struct *self, int x)
{
    return (self->total >= x) ? true : false;
}
```




# Implementation of Spying

```C:spy_hoge.c
// OUT の private データ構造体(struct hoge_struct)
#include "hoge_private.h"

typedef struct spy_hoge_struct {
    struct hoge_struct orig;
    // 呼び出し履歴記録
} *spy_hoge;
```


# Mocking
